In [1]:
import producer,consumer
import pandas as pd
import numpy as np 
from pmdarima.arima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [2]:
def process(file_path):

    df =  pd.read_csv(file_path)
    df = df.drop('AccountOID', axis=1)
    dates = df['ConsumptionDate'].to_list()
    all_dates = pd.date_range(dates[0], dates[-1])
    all_dates_df = pd.DataFrame(pd.date_range(dates[0], dates[-1]), columns=['ConsumptionDate'])
    df['ConsumptionDate'] = pd.to_datetime(df['ConsumptionDate'])
    df = pd.merge(all_dates_df, df, on='ConsumptionDate', how='left')
    df['Consumption'] = df['Consumption'].fillna(0)
    df["ConsumptionDate"] = pd.to_datetime(df["ConsumptionDate"])
    df.loc[df['Consumption'] > 800, 'Consumption'] = df['Consumption'].median()
    df['ConsumptionDate'] = df['ConsumptionDate'].astype(str)
    df.reset_index(drop=True, inplace=True)
    return df


data_processed = process("dataset.csv")

data_processed.shape[0]

103841

In [3]:
df = data_processed.to_dict('records')
data_processed.dtypes
# print(df)

ConsumptionDate     object
Consumption        float64
dtype: object

In [4]:
%%time
producer.main(data_processed)


[Error 3] UnknownTopicOrPartitionError: Request 'DeleteTopicsRequest_v3(topics=['stable_topic'], timeout=30000)' failed with response 'DeleteTopicsResponse_v3(throttle_time_ms=0, topic_error_codes=[(topic='stable_topic', error_code=3)])'.
CPU times: user 32.8 s, sys: 3.4 s, total: 36.2 s
Wall time: 40.2 s


In [5]:
%%time
training_data = consumer.main()
training_data_df = pd.DataFrame.from_records(training_data)
training_data_df['ConsumptionDate'] = pd.to_datetime(training_data_df['ConsumptionDate'])
training_data_df = training_data_df.set_index('ConsumptionDate')
training_data_df


CPU times: user 1.28 s, sys: 16.8 ms, total: 1.3 s
Wall time: 6.01 s


,Consumption
ConsumptionDate,
2020-02-22,11.070
2020-02-22,0.000
2020-02-22,0.000
2020-02-22,0.153
2020-02-22,1.461
...,...
2021-03-20,0.712
2021-03-20,0.353
2021-03-20,0.080


In [6]:
def modeling(dataset):

    data_size = dataset.shape[0]
    test_size = 30
    train_X, train_y = dataset[:data_size - test_size], dataset[:data_size - test_size]
    test_X, test_y = dataset[data_size - test_size:], dataset[data_size - test_size:]

    #Automating ARIMA to choose best parameters    
    step_wise = auto_arima(train_y,exogenous= train_X, seasonal=True,start_p=1, start_q=1, max_p=7, max_q=7,
    error_action = 'ignore', suppress_warnings = True, stepwise=True, parallel = True, verbose=False)

    #SARIMAX modeling with the choosen parameters  
    model= SARIMAX(train_y,exog=train_X,order=step_wise.order)
    results = model.fit()
    #Predictions
    predictions = results.predict(start=data_size - test_size, end=data_size - 1, exog=np.array(test_X))

    return predictions

In [7]:
%%time
modeling(training_data_df)

/home/raed/code/assigment/venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/raed/code/assigment/venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -1.05940D+01    |proj g|=  1.43613D+08

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    2      1     21      1     0     0   1.436D+08  -1.059D+01
  F =  -10.593986931765556     

ABNORMAL_TERMINATION_IN_LNSRCH                              



 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.
/home/raed/code/assigment/venv/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


CPU times: user 1min 17s, sys: 3min, total: 4min 18s
Wall time: 25.9 s


/home/raed/code/assigment/venv/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


103811      0.000
103812      0.001
103813      0.000
103814      9.030
103815      0.952
103816     46.915
103817      0.122
103818      9.202
103819      1.761
103820      0.515
103821      0.453
103822      7.369
103823      0.519
103824      0.514
103825      0.117
103826      0.000
103827    145.761
103828      4.903
103829      0.000
103830      0.077
103831      0.273
103832     17.895
103833    548.600
103834      0.886
103835      5.089
103836      0.712
103837      0.353
103838      0.080
103839      1.060
103840      0.000
Name: predicted_mean, dtype: float64